# Week9 Homework
## Working with Strings

The data frame contains notes taken by staff members. Your task is to clean it up to make it usable for analysis. 

### Tasks
* Create start date, end date columns and populate the values (If notes don't include month or day use 1) i.e: 2018 => 2018-01-01

* Create phone number column (If notes don't include phone numbers, use None)
* Create a column `duration` showing difference between start & end dates in years as a whole number

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS601_Fall21/main/Week09/data/hw_data.csv', index_col=0)
df.head()

,email,notes
0,fosterjennifer@gmail.com,st Dt February 2005 ed dt;10/27/2011
1,vanessastephens@gmail.com,st date:1974-05-21 end date 06/17/2013 pn: 00...
2,vsmith@yahoo.com,start dt 01/14/1985-ed date:2021-03-21
3,tammymorgan@gmail.com,STDT July 1973-end dt 01/20/2012
4,kimberlylyons@mason-solis.biz,st date:09/28/2002-ENDT 04/10-p/n 929-878-4923


In [3]:
# pattern to find phone number with extension
pattern = re.compile(r'\(?([0-9]{3})\)?([ .-]?)([0-9]{3})([ .-]?)([0-9]{4})(x([0-9]{1,}))?')

In [4]:
# created phone number column and assigned output if it matches with regex otherwise assigned none
df['phone_number'] = [pattern.search(s).group() if pattern.search(str(s)) != None else None for s in df['notes']]
df.head()

,email,notes,phone_number
0,fosterjennifer@gmail.com,st Dt February 2005 ed dt;10/27/2011,None
1,vanessastephens@gmail.com,st date:1974-05-21 end date 06/17/2013 pn: 00...,186-444-5580x968
2,vsmith@yahoo.com,start dt 01/14/1985-ed date:2021-03-21,None
3,tammymorgan@gmail.com,STDT July 1973-end dt 01/20/2012,None
4,kimberlylyons@mason-solis.biz,st date:09/28/2002-ENDT 04/10-p/n 929-878-4923,929-878-4923


In [5]:
# for loop to extract start and end dates(popping out phonenumber selected text from list)
pattern2 = re.compile(r'^[0-9]{4}')
list=[]
for i in range(0,len(df['notes'])):
    substring = [x.group() for x in re.finditer( r'([0-9]{2}(-|\/)[0-9]{2}(-|\/)[0-9]{4})|([0-9]{4}(-|\/)[0-9]{2}(-|\/)[0-9]{2})|((January|February|March|April|May|June|July|August|September|October|November|December) [0-9]{4})|([0-9]{2}(\/)[0-9]{2})|(\b[0-9]{4}\b)(-[0-9]{2})?', df['notes'][i])]
    if(len(substring)>2):
        for i in range(0,len(substring)-1):
            x = pattern2.findall(substring[i])
            if len(x)!=0:
                if ((int(x[0])<1900) | (int(x[0])>2021)):
                    substring.remove(x[0])
    if(len(substring)>2):
        substring.pop()
    list.append(substring)
list
            
df['Dates'] = list

In [6]:
import datetime
# created start and end date columns and assigned values from list
df[['Start_Date','End_Date']] = pd.DataFrame(df['Dates'].to_list(), index=df.index)
pattern3 = re.compile(r'(\b^[0-9]{2}(\/)[0-9]{2})$\b')
# formated start and end date columns into date fields (if the format is in mm/yy format converted them separately)
df['Formated_Start_Date'] = [pd.to_datetime(s,format='%m/%y').date() if pattern3.search(str(s)) != None else pd.to_datetime(s,errors='coerce').date() for s in df['Start_Date']]
df['Formated_End_Date'] = [pd.to_datetime(s,format='%m/%y').date() if pattern3.search(str(s)) != None else pd.to_datetime(s,errors='coerce').date() for s in df['End_Date']]
df['Formated_Start_Date'] = pd.to_datetime(df['Formated_Start_Date'])
df['Formated_End_Date'] = pd.to_datetime(df['Formated_End_Date'])
df.head()

,email,notes,phone_number,Dates,Start_Date,End_Date,Formated_Start_Date,Formated_End_Date
0,fosterjennifer@gmail.com,st Dt February 2005 ed dt;10/27/2011,None,"[February 2005, 10/27/2011]",February 2005,10/27/2011,2005-02-01,2011-10-27
1,vanessastephens@gmail.com,st date:1974-05-21 end date 06/17/2013 pn: 00...,186-444-5580x968,"[1974-05-21, 06/17/2013]",1974-05-21,06/17/2013,1974-05-21,2013-06-17
2,vsmith@yahoo.com,start dt 01/14/1985-ed date:2021-03-21,None,"[01/14/1985, 2021-03-21]",01/14/1985,2021-03-21,1985-01-14,2021-03-21
3,tammymorgan@gmail.com,STDT July 1973-end dt 01/20/2012,None,"[July 1973, 01/20/2012]",July 1973,01/20/2012,1973-07-01,2012-01-20
4,kimberlylyons@mason-solis.biz,st date:09/28/2002-ENDT 04/10-p/n 929-878-4923,929-878-4923,"[09/28/2002, 04/10]",09/28/2002,04/10,2002-09-28,2010-04-01


In [7]:
# created duration column and subtracted start_date year from end_date year
df['Duration'] = pd.DatetimeIndex(df['Formated_End_Date']).year - pd.DatetimeIndex(df['Formated_Start_Date']).year
df.head()

,email,notes,phone_number,Dates,Start_Date,End_Date,Formated_Start_Date,Formated_End_Date,Duration
0,fosterjennifer@gmail.com,st Dt February 2005 ed dt;10/27/2011,None,"[February 2005, 10/27/2011]",February 2005,10/27/2011,2005-02-01,2011-10-27,6
1,vanessastephens@gmail.com,st date:1974-05-21 end date 06/17/2013 pn: 00...,186-444-5580x968,"[1974-05-21, 06/17/2013]",1974-05-21,06/17/2013,1974-05-21,2013-06-17,39
2,vsmith@yahoo.com,start dt 01/14/1985-ed date:2021-03-21,None,"[01/14/1985, 2021-03-21]",01/14/1985,2021-03-21,1985-01-14,2021-03-21,36
3,tammymorgan@gmail.com,STDT July 1973-end dt 01/20/2012,None,"[July 1973, 01/20/2012]",July 1973,01/20/2012,1973-07-01,2012-01-20,39
4,kimberlylyons@mason-solis.biz,st date:09/28/2002-ENDT 04/10-p/n 929-878-4923,929-878-4923,"[09/28/2002, 04/10]",09/28/2002,04/10,2002-09-28,2010-04-01,8


In [8]:
#Final DF (selected required columns from old df)
df_finaldf = pd.DataFrame()
df_finaldf[['email','notes','Start_Date','End_Date','Phone_Number','Duration']] = df[['email','notes','Formated_Start_Date','Formated_End_Date','phone_number','Duration']]
df_finaldf.head(15)

,email,notes,Start_Date,End_Date,Phone_Number,Duration
0,fosterjennifer@gmail.com,st Dt February 2005 ed dt;10/27/2011,2005-02-01,2011-10-27,None,6
1,vanessastephens@gmail.com,st date:1974-05-21 end date 06/17/2013 pn: 00...,1974-05-21,2013-06-17,186-444-5580x968,39
2,vsmith@yahoo.com,start dt 01/14/1985-ed date:2021-03-21,1985-01-14,2021-03-21,None,36
3,tammymorgan@gmail.com,STDT July 1973-end dt 01/20/2012,1973-07-01,2012-01-20,None,39
4,kimberlylyons@mason-solis.biz,st date:09/28/2002-ENDT 04/10-p/n 929-878-4923,2002-09-28,2010-04-01,929-878-4923,8
5,cooperjason@hotmail.com,st Dt 2001-04 ENDT 2013-04 p# 8400434845,2001-04-01,2013-04-01,8400434845,12
6,kristywise@gmail.com,pn: 120-577-2777-start dt June 1980-ENDT 2017-01,1980-06-01,2017-01-01,120-577-2777,37
7,scottstacie@romero-obrien.org,p/n 773.229.8494/st dt 1998-06/end Dt 03/21,1998-06-01,2021-03-01,773.229.8494,23
8,careyrobert@horn-spencer.com,start dt 1986/ENDT 03/18/2014,1986-01-01,2014-03-18,None,28
9,mcole@riley-ho.biz,STDT 08/89 ed date:March 2019 p# 3855702012,1989-08-01,2019-03-01,3855702012,30
